In [1]:
pwd()

'c:\\Users\\asus\\Documents\\Projects\\ECommerce Customer Churn Prediction\\notebooks'

In [2]:
import os
os.chdir("../")

In [3]:
import pandas as pd
import numpy as np
import pyodbc
from sklearn.preprocessing import StandardScaler
from src.configuration.config import ConfigurationManager



In [8]:
import os
import pandas as pd
import pyodbc
from src.logger import logger
import sqlite3

class DataTransformationStorage:
    def __init__(self):
        self.config=ConfigurationManager().get_data_storage_config()
        #self.data_config=ConfigurationManager().get_data_ingestion_config()
        print(self.config)
    def store_in_sql(self,df):
        # SQLite Database Setup
        conn = sqlite3.connect(self.config.db_path)
        cursor = conn.cursor()

        # Create Table for Transformed Data
        cursor.execute(f'''
        CREATE TABLE IF NOT EXISTS {self.config.data_table_name} (
            CustomerID INTEGER PRIMARY KEY,
            Churn INTEGER,
            Tenure REAL,
            NumberOfDeviceRegistered INTEGER,
            PreferedOrderCat INTEGER,
            SatisfactionScore INTEGER,
            MaritalStatus INTEGER,
            Complain INTEGER,
            DaySinceLastOrder REAL,
            CashbackAmount REAL,
            TotalSpend REAL,
            ActivityFrequency REAL,
            IsLongTermCustomer INTEGER
        );
        ''')

        # Insert Transformed Data into Database
        df.to_sql(self.config.data_table_name, conn, if_exists="replace", index=False)
        conn.close()
    
    def data_transformation(self):
        
        df=pd.read_csv(self.config.processed+"churn_data_prepared.csv")
        # Feature Engineering
        # Aggregated Feature: Total spend per customer (Assuming CashbackAmount as a proxy for spend)
        df["TotalSpend"] = df["CashbackAmount"] * df["Tenure"]

        # Derived Feature: Activity Frequency
        df["ActivityFrequency"] = df["NumberOfDeviceRegistered"] / df["Tenure"].replace(0, 1)
        
        self.store_in_sql(df)
        
        
        

In [10]:
obj=DataTransformationStorage()
obj.data_transformation()

[2025-03-01 13:10:17,064 : INFO: common: yaml file config\config.yaml loaded successfully!]
[2025-03-01 13:10:17,080 : INFO: common: yaml file config\params.yaml loaded successfully!]
[2025-03-01 13:10:17,096 : INFO: common: yaml file config\schema.yaml loaded successfully!]
DataStorageConnectionConfig(db_path='config/churn_data.db', processed='data/processed/', data_table_name='transformed_churn_data')
